In [1]:
import requests

URL = "https://www.12371.cn/special/xxzd/jh/"
page = requests.get(URL)
# print(page.text)

In [2]:
# Find all URLs in page

import re

# Define the regular expression pattern to match URLs
url_pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

# Find all URLs in the page text
urls = re.findall(url_pattern, page.text)

In [6]:
# Filter URLs to leave only the speech articles

filtered_urls = [url for url in urls if "news.12371.cn" in url and "ARTI" in url]
filtered_urls = [url.replace("','title':'", "") for url in filtered_urls]

filtered_urls = list(set(filtered_urls))


In [7]:
# Write filtered URLs to a file

with open('12371_urls.txt', 'w') as file:
    for url in filtered_urls:
        file.write(url + '\n')

In [14]:
# Read URLs from file
urls = []
with open('12371_urls.txt', 'r') as file:
    for line in file:
        url = line.strip()
        urls.append(url)

In [13]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm

directory = r'text_output/12371'
if not os.path.exists(directory):
    os.makedirs(directory)

def save_text(urls):
    errURLs = []
    for url in tqdm(urls):
        try:
            page = requests.get(url)
            page.encoding = page.apparent_encoding

            # Create a BeautifulSoup object with the page content
            soup = BeautifulSoup(page.text, 'html.parser')

            meta = soup.find('i', class_="time").text

            title = soup.find('h1', class_="big_title").text.replace('\n', ' ').strip()

            body_text = ''.join([element.text for element in soup.find_all('div', class_='con') if element.text.strip()])

            if '延伸阅读' in body_text:
                body_text = body_text.split('延伸阅读')[0]

            file_path = f'{directory}/{title}.txt'
            i = 1
            while os.path.exists(file_path):
                modified_title = f'{title}_{i}'
                file_path = f'{directory}/{modified_title}.txt'
                i += 1
            
            with open(file_path, 'w') as file:
                file.write(meta + '\n\n')
                file.write(title + '\n\n')
                file.write(body_text)
        
        except:
            print(f"Error processing {url}")
            errURLs.append(url)
            continue
        
    return errURLs



In [15]:
errURLs = save_text(urls)
save_text(errURLs)

100%|██████████| 589/589 [05:02<00:00,  1.95it/s]
0it [00:00, ?it/s]


[]